In [1]:
import pandas as pd


def merge_link_parsed_with_api(
    link_parsed_path: str,
    filtered_api_path: str,
    output_path: str,
):
    """
    link_parsed.csv(크롤링 결과) + filtered_api_data.csv(API 필터링 결과)를
    하나의 스키마로 맞춘 뒤(concat) 단일 파일로 저장합니다.

    ✅ 기준(바탕): link_parsed.csv
    ✅ API 데이터 매핑:
        - 대상   = "청년"
        - 정책명 = servNm
        - 요약   = servDgst
        - 링크   = servDtlLink
    """

    # -----------------------------
    # 1) 크롤링 결과 로드 (바탕)
    # -----------------------------
    # utf-8-sig로 저장한 파일이 많아 우선 utf-8-sig로 읽고,
    # 실패하면 utf-8로 한 번 더 시도하는 방식도 가능하지만
    # 여기서는 깔끔하게 utf-8-sig로 고정합니다.
    df_base = pd.read_csv(link_parsed_path, encoding="utf-8-sig")

    # link_parsed.csv는 보통 ['대상','정책명','요약','링크']를 가지고 있다고 가정합니다.
    # 혹시 컬럼명이 다르면 아래 print로 확인해서 맞춰주세요.
    # print(df_base.columns)

    # -----------------------------
    # 2) API 결과 로드
    # -----------------------------
    df_api_raw = pd.read_csv(filtered_api_path, encoding="utf-8-sig")
    # print(df_api_raw.columns)

    # -----------------------------
    # 3) API 결과를 link_parsed 스키마로 변환
    # -----------------------------
    # 필요한 컬럼이 없을 때를 대비해 get을 쓰고,
    # 없는 경우는 빈 문자열로 처리합니다.
    df_api = pd.DataFrame({
        "대상": "청년",
        "정책명": df_api_raw.get("servNm", "").fillna(""),
        "요약": df_api_raw.get("servDgst", "").fillna(""),
        "링크": df_api_raw.get("servDtlLink", "").fillna(""),
    })

    # -----------------------------
    # 4) 병합 (concat)
    # -----------------------------
    # link_parsed를 “바탕”으로 하므로 base를 앞에 두고 뒤에 API를 붙입니다.
    df_merged = pd.concat([df_base, df_api], ignore_index=True)

    # -----------------------------
    # 5) (권장) 중복 제거
    # -----------------------------
    # 정책이 겹칠 수 있으니 링크 기준으로 1차 중복 제거 권장
    # 링크가 비어있다면 정책명+요약 조합으로 중복 제거 보완
    df_merged["링크"] = df_merged["링크"].fillna("").astype(str).str.strip()
    df_merged["정책명"] = df_merged["정책명"].fillna("").astype(str).str.strip()
    df_merged["요약"] = df_merged["요약"].fillna("").astype(str).str.strip()

    # 링크가 있는 행은 링크 기준으로 중복 제거
    has_link = df_merged["링크"] != ""
    df_with_link = df_merged[has_link].drop_duplicates(subset=["링크"], keep="first")

    # 링크가 없는 행은 (정책명, 요약) 기준으로 중복 제거
    df_no_link = df_merged[~has_link].drop_duplicates(subset=["정책명", "요약"], keep="first")

    df_merged = pd.concat([df_with_link, df_no_link], ignore_index=True)

    # -----------------------------
    # 6) 저장
    # -----------------------------
    df_merged = df_merged.dropna(axis=1, how="all")
    df_merged.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"✅ 병합 완료: {output_path}")
    print(f"- merged (dedup 적용 후): {len(df_merged)}건")

    return df_merged


if __name__ == "__main__":
    merged_df = merge_link_parsed_with_api(
        link_parsed_path="link_parsed.csv",
        filtered_api_path="filtered_api_data.csv",
        output_path="merged_policies.csv",
    )

    # 상위 5개 확인
    print(merged_df.head(5))


✅ 병합 완료: merged_policies.csv
- merged (dedup 적용 후): 44건
     대상                     정책명  \
0  신혼부부  신혼부부 임차보증금 반환보증 보증료 지원   
1  신혼부부       전세보증금 반환보증 보증료 지원   
2  신혼부부         신혼부부 임차보증금 이자지원   
3  신혼부부      건물 에너지 효율화 사업 융자지원   
4  신혼부부             주택개량 신축융자지원   

                                                  요약  \
0  신혼부부 임차보증금 이자지원 신규 대출자 대상(’24.7.30. 이후) 임차보증금 ...   
1  전세 피해 예방을 위한 전세보증금 반환보증료 지원사업의 대상이 청년에서 전 연령층으...   
2  신혼부부들의 행복한 미래를 위해 더 나은 주거환경과 소득대비 높은 주거비 부담을 완...   
3  에너지 이용 효율을 높이고 온실가스를 감축하기 위해 건물 단열공사, LED 조명교체...   
4  저층주거지 내 노후주택 집수리 융자 지원 집수리 공사비용의 일부를 지원하여 저층주거...   

                                                  링크  
0  https://housing.seoul.go.kr/site/main/content/...  
1  https://housing.seoul.go.kr/site/main/content/...  
2  https://housing.seoul.go.kr/site/main/content/...  
3  https://housing.seoul.go.kr/site/main/content/...  
4  https://housing.seoul.go.kr/site/main/content/...  
